<a href="https://colab.research.google.com/github/wallik2/Bayesian_Network/blob/main/Bayesian_Network_%5BProbability_of_Depression_%26_OCD%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pomegranate

     |████████████████████████████████| 4.3 MB 25.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pomegranate: filename=pomegranate-0.14.8-cp37-cp37m-linux_x86_64.whl size=15006349 sha256=41a2eb821059c73b3601a2d0a1ffad5f53c91a5100b23526485bcb95abfe958c
  Stored in directory: /root/.cache/pip/wheels/24/68/69/0eaab474ef1d65abedcd47de8a38ab21d221d329954d7edd24
Successfully built pomegranate


In [4]:
from pomegranate import *

In [31]:
depression = DiscreteDistribution({'True':0.067, 'False':0.933})
#ocd = DiscreteDistribution({'True':0.5, 'False':0.5})


panic_attack_D = ConditionalProbabilityTable(
                [[ 'True', 'True', 0.6 ],
                  [ 'True', 'False', 0.08 ],
                  [ 'False', 'True', 0.4 ],
                  [ 'False', 'False', 0.92 ]], [depression])

intrusive_thought_D = ConditionalProbabilityTable(
                    [[ 'True', 'True', 0.75 ],
                      [ 'True', 'False', 0.08 ],
                      [ 'False', 'True', 0.25 ],
                      [ 'False', 'False', 0.92 ]], [depression])

worthless_D = ConditionalProbabilityTable(
                [[ 'True', 'True', 0.99 ],
                  [ 'True', 'False', 0.005 ],
                  [ 'False', 'True', 0.01 ],
                  [ 'False', 'False', 0.995 ]], [depression])

In [32]:
ocd = DiscreteDistribution({'True':0.5, 'False':0.5})

panic_attack_O = ConditionalProbabilityTable(
                [[ 'True', 'True', 0.75 ],
                  [ 'True', 'False', 0.01 ],
                  [ 'False', 'True', 0.25 ],
                  [ 'False', 'False', 0.99 ]], [ocd])

intrusive_thought_O = ConditionalProbabilityTable(
                    [[ 'True', 'True', 0.95 ],
                      [ 'True', 'False', 0.02 ],
                      [ 'False', 'True', 0.05 ],
                      [ 'False', 'False', 0.98 ]], [ocd])

worthless_O = ConditionalProbabilityTable(
                [[ 'True', 'True', 0.8 ],
                  [ 'True', 'False', 0.1 ],
                  [ 'False', 'True', 0.2 ],
                  [ 'False', 'False', 0.9 ]], [ocd])

In [33]:
d0 = State( depression, name= "depression")
d1 = State( panic_attack_D, name= "panic attack")
d2 = State( intrusive_thought_D, name= "intrusive thought")
d3 = State( worthless_D, name= "feel worthless")

network_depression = BayesianNetwork( "Depression Mental Diagnosis" )

network_depression.add_nodes(d0, d1, d2, d3)
network_depression.add_edge(d0, d1)
network_depression.add_edge(d0, d2)
network_depression.add_edge(d0, d3)

network_depression.bake()

In [37]:
o0 = State( ocd, name= "ocd")
o1 = State( panic_attack_O, name= "panic attack")
o2 = State( intrusive_thought_O, name= "intrusive thought")
o3 = State( worthless_O, name= "feel worthless")

network_ocd = BayesianNetwork( "OCD Mental Diagnosis" )

network_ocd.add_nodes(o0, o1, o2, o3)
network_ocd.add_edge(o0, o1)
network_ocd.add_edge(o0, o2)
network_ocd.add_edge(o0, o3)

network_ocd.bake()

In [55]:
observations = { 'panic attack':'True','intrusive thought':'False','feel worthless':'True'}
beliefs = network_depression.predict_proba(observations)


for state, belief in zip( network.states,beliefs):
  print(state.name, belief)

depression {
    "class" : "Distribution",
    "dtype" : "str",
    "name" : "DiscreteDistribution",
    "parameters" : [
        {
            "True" : 0.48114028724793556,
            "False" : 0.5188597127520644
        }
    ],
    "frozen" : false
}
panic attack True
intrusive thought False
feel worthless True


---

In [53]:
def mental_patient_interview():
    symptoms_list = ['panic attack','intrusive thought',"feel worthless"]
    observations = {}
    for symptom in symptoms_list:
        ans = input('Do you have a ' + symptom + ': ')
        if ans == 'yes':
            observations[symptom]= 'True'
        elif ans == 'no':
            observations[symptom]= 'False'
            
    #print(observations)
    beliefs_depression = network_depression.predict_proba(observations)
    beliefs_ocd = network_ocd.predict_proba(observations)
    print('\n\n')
  
    for state, belief in zip(network_depression.states, beliefs_depression):
        if not isinstance(belief, str) :
            print( state.name,': probability = ',belief.probability('True'))
    
    for state, belief in zip(network_ocd.states, beliefs_ocd):
        if not isinstance(belief, str) :
            print( state.name,': probability = ',belief.probability('True'))


In [54]:
mental_patient_interview()

Do you have a panic attack: yes
Do you have a intrusive thought: no
Do you have a feel worthless: yes



depression : probability =  0.48114028724793556
ocd : probability =  0.19672131147541183
